In [ ]:
import sys
# set CUDA_LAUNCH_BLOCKING=1
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
# import imageio.v3 as imageio
# import napari

# from micro_sam import instance_segmentation, util
# from micro_sam.multi_dimensional_segmentation import automatic_3d_segmentation
# from disentangle.core.tiff_reader import load_tiff

# def cell_segmentation():
#     """Run the instance segmentation functionality from micro_sam for segmentation of
#     HeLA cells. You need to run examples/annotator_2d.py:hela_2d_annotator once before
#     running this script so that all required data is downloaded and pre-computed.
#     """
#     image_path = "/home/ashesh.ashesh/code/Disentangle/disentangle/notebooks/test_img.tiff"
#     embedding_path = "../embeddings/embeddings-hela2d.zarr"

#     # Load the image, the SAM Model, and the pre-computed embeddings.
#     image = load_tiff(image_path)
#     predictor = util.get_sam_model()
#     embeddings = util.precompute_image_embeddings(predictor, image, save_path=embedding_path)

#     # Use the instance segmentation logic of Segment Anything.
#     # This works by covering the image with a grid of points, getting the masks for all the poitns
#     # and only keeping the plausible ones (according to the model predictions).
#     # While the functionality here does the same as the implementation from Segment Anything,
#     # we enable changing the hyperparameters, e.g. 'pred_iou_thresh', without recomputing masks and embeddings,
#     # to support (interactive) evaluation of different hyperparameters.

#     # Create the automatic mask generator class.
#     amg = instance_segmentation.AutomaticMaskGenerator(predictor)

#     # Initialize the mask generator with the image and the pre-computed embeddings.
#     amg.initialize(image, embeddings, verbose=True)

#     # Generate the instance segmentation. You can call this again for different values of 'pred_iou_thresh'
#     # without having to call initialize again.
#     instances = amg.generate(pred_iou_thresh=0.88)
#     # instances = instance_segmentation.mask_data_to_segmentation(
#     #     instances, with_background=True
#     # )

#     # # instances = instance_segmentation.mask_data_to_segmentation(
#     # #     instances, shape=image.shape, with_background=True
#     # # )

#     # # Show the results.
#     # v = napari.Viewer()
#     # v.add_image(image)
#     # v.add_labels(instances)
#     # napari.run()
#     return image, instances


# def cell_segmentation_with_tiling():
#     """Run the instance segmentation functionality from micro_sam for segmentation of
#     cells in a large image. You need to run examples/annotator_2d.py:wholeslide_annotator once before
#     running this script so that all required data is downloaded and pre-computed.
#     """
#     image_path = "../data/whole-slide-example-image.tif"
#     embedding_path = "../embeddings/whole-slide-embeddings.zarr"

#     # Load the image, the SAM Model, and the pre-computed embeddings.
#     image = imageio.imread(image_path)
#     predictor = util.get_sam_model()
#     embeddings = util.precompute_image_embeddings(
#         predictor, image, save_path=embedding_path, tile_shape=(1024, 1024), halo=(256, 256)
#     )

#     # Use the instance segmentation logic of Segment Anything.
#     # This works by covering the image with a grid of points, getting the masks for all the poitns
#     # and only keeping the plausible ones (according to the model predictions).
#     # The functionality here is similar to the instance segmentation in Segment Anything,
#     # but uses the pre-computed tiled embeddings.

#     # Create the automatic mask generator class.
#     amg = instance_segmentation.TiledAutomaticMaskGenerator(predictor)

#     # Initialize the mask generator with the image and the pre-computed embeddings.
#     amg.initialize(image, embeddings, verbose=True)

#     # Generate the instance segmentation. You can call this again for different values of 'pred_iou_thresh'
#     # without having to call initialize again.
#     instances = amg.generate(pred_iou_thresh=0.88)
#     instances = instance_segmentation.mask_data_to_segmentation(
#         instances, shape=image.shape, with_background=True
#     )

#     # Show the results.
#     v = napari.Viewer()
#     v.add_image(image)
#     v.add_labels(instances)
#     v.add_labels(instances)
#     napari.run()


In [ ]:
import numpy as np
# import matplotlib.pyplot as plt
from cellpose import models, io
from cellpose.io import imread

io.logger_setup()

# model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
model = models.Cellpose(model_type='nuclei', gpu=True)

# list of files
# PUT PATH TO YOUR FILES HERE!
files = ['/home/ashesh.ashesh/code/Disentangle/disentangle/notebooks/test_img.tiff']

imgs = [imread(f) for f in files]
nimg = len(imgs)

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
channels = [[0,0]]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended)
# diameter can be a list or a single number for all images

masks, flows, styles, diams = model.eval(imgs, diameter=None, channels=channels)


### or to run one of the other models, or a custom model, specify a CellposeModel
# model = models.CellposeModel(model_type='livecell_cp3')

# masks, flows, styles = model.eval(imgs, diameter=30, channels=[0,0])

In [ ]:
%%timeit 
masks, flows, styles, diams = model.eval(imgs, diameter=None, channels=channels)

In [ ]:
import pandas as pd
pd.Series(masks[0].flatten()).value_counts().sort_index()
# np.unique(masks[0])

In [ ]:
import matplotlib.pyplot as plt
_,ax = plt.subplots(figsize=(10,5),ncols=2)
ax[0].imshow(masks[0])
ax[1].imshow(imgs[0])

In [ ]:
import torch

In [ ]:
from cellpose import models
from disentangle.core.tiff_reader import load_tiff

model = models.Cellpose(gpu=True, model_type='nuclei')
channels = [0,0]
imgs_2D =  load_tiff("/home/ashesh.ashesh/code/Disentangle/disentangle/notebooks/test_img.tiff")


In [ ]:
imgs_2D = imgs_2D[None]

In [ ]:
imgs_2D = imgs_2D[:512, :512]

In [ ]:
masks, flows, styles, diams = model.eval([imgs_2D], diameter=None, flow_threshold=None, channels=channels)